# **Implement LSH**

## Group members : 
#### 1. Mintra Sojiphan 6220422057 DS6
#### 2. Kavin Singhakhet 6310422040 DS7
#### 3.

### **We implement LSH using two packages with two differnt datasets**
#### 1. Package: SnaPy // Datasets: A Corpus of Plagiarised Short Answers 
####    References: https://github.com/justinnbt/SnaPy 
####                https://ir.shef.ac.uk/cloughie/resources/plagiarism_corpus.html

#### 2. Package: datasketch // Dataset: News headlines
####    References: https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/


### **1. Package: SnaPy // Datasets: A Corpus of Plagiarised Short Answers**
### We will implement LSH on plagiarised answers of five different questions namely question A, B, C, D, and E to identify near duplicate answers of each question using Jaccard similarity threshold (s) = 0.5



### Step 1: Install and import packages



In [ ]:
pip install snapy

In [ ]:
pip install mmh3

In [ ]:
from snapy import MinHash, LSH
from google.colab import drive
import matplotlib.pyplot as plt
import networkx as nx

###Step 2: Connect Google Colab with Google drive

In [ ]:
drive.mount("/content/drive")

### Step 3: Create function to perform LSH

In [ ]:
 def LSH_task(task):

  #import data 
  files=[]
  x='/content/drive/MyDrive/corpus/'+task+'.txt'
  files.append(x)
  for i in range(1,20):
    z='/content/drive/MyDrive/corpus/'+task+ ' ('+str(i)+').txt'
    files.append(z)

  docs=[]
  for file in files:
    file = open(file)
    text = file.read()
    docs.append(text)
  file.close()

  print(docs)
  print()
  print('#task: ',len(docs))
  labels=[]
  labels.append(task)
  for i in range(1,20):
    a=task+'('+str(i)+')'
    labels.append(a)

  seed =3

  #Create MinHash object.
  minhash = MinHash(docs, n_gram=9, permutations=100, hash_bits=64, seed=3)
  print('Signatures metric: ',len(minhash.signatures),'x',len(minhash.signatures[0]))
  print('#permutations used to create signatures:',minhash.permutations)
  #print('Minhash Signatures for each text:')
  #for i in minhash.signatures:
    #print(i)
  
  # Create LSH model.
  lsh = LSH(minhash, labels, no_of_bands=50)

  # Query to find near duplicates for each doc.
  print()
  for i in labels:
    print('Near duplicate for answer',i,':',lsh.query(i, min_jaccard=0.1))

  # Check contents of documents.
  print(lsh.contains())

  # Return adjacency list for all similar texts.
  adjacency_list = lsh.adjacency_list(min_jaccard=0.1)
  print()
  print('adjacency_lists: ',adjacency_list)
  print()
  # Returns edge list for use creating a weighted graph.
  edge_list = lsh.edge_list(min_jaccard=0.1, jaccard_weighted=True)
  print('edge lists: ',edge_list)
  print()

  # Create Undirected weighted graph.
  fig=plt.figure(figsize =(12, 7))
  fig.set_facecolor("#181818")
  title="Near duplicate answer of question "+task
  fig.suptitle(title,color= '#cccccc',fontsize=25)
  G = nx.Graph()
  for i in edge_list:
    G.add_edge(i[0],i[1],weight=i[2])
  e1=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] > 0.5]
  e2=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] <= 0.5]
  pos=nx.spring_layout(G) 

  # nodes
  nx.draw_networkx_nodes(G,pos,node_size=1300,node_color='yellow')

  # edges
  edge1=nx.draw_networkx_edges(G,pos,edgelist=e1,width=4,edge_color='red')
  edge2=nx.draw_networkx_edges(G,pos,edgelist=e2,width=1,edge_color='red',style='dashed')

  # labels
  nx.draw_networkx_labels(G,pos,font_family='sans-serif',font_size=12,font_color='#000000',font_weight=100)
  keys=[(i[0],i[1]) for i in edge_list]
  values= [(i[2]) for i in edge_list]
  edge_labels = dict(zip(keys, values))
  nx.draw_networkx_edge_labels(G,pos,edge_labels= edge_labels,font_color='red')                          
  fig.set_facecolor("#181818")
  plt.axis('off')
  fig.legend((edge1, edge2), ('sim(i,j) > 0.5', 'sim(i,j) <= 0.5'),loc=1,fontsize=15)
  plt.savefig("LSH_graph.png")
  plt.show()

### Step 4: Perform LSH on set of 20 plagiarised answers of the following questions to identify near duplicate answers using Jaccard similarity threshold (s) = 0.5



#### **Question A**: The result illustrates that there are 3 pairs of answers which can be identified as near duplicate answers as shown below:
#### 1. Answer A(1) and A(18) with Jaccard similarity of 0.9
#### 2. Answer A(1) and A(6) with Jaccard similarity of 0.74
#### 3. Answer A(6) and A(18) with Jaccard similarity of 0.7

In [ ]:
LSH_task('A')

#### **Question B**: The result illustrates that there are no pairs of answers which can be identified as near duplicate answers

In [ ]:
LSH_task('B')

#### **Question C**: The result illustrates that there are no pairs of answers which can be identified as near duplicate answers

In [ ]:
LSH_task('C')

#### **Question D**:The result illustrates that there are 3 pairs of answers which can be identified as near duplicate answers as shown below:
#### 1. Answer D and D(14) with Jaccard similarity of 0.7
#### 2. Answer D and D(18) with Jaccard similarity of 0.68
#### 3. Answer D(18) and A(14) with Jaccard similarity of 0.62

In [ ]:
LSH_task('D')

#### **Question E**: The result illustrates that there are no pairs of answers which can be identified as near duplicate answers

In [ ]:
LSH_task('E')

### **2. Package: datasketch // Dataset: News headlines**
### We will implement LSH to create news headlines recommendation 

### Step 1: Install and Import packages

In [1]:
pip install datasketch

     |████████████████████████████████| 71kB 5.2MB/s 


In [2]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

### Step 2: Acquire data using web scraping package (BeautifulSoup)

In [11]:
from bs4 import BeautifulSoup
import requests

url1 = 'https://www.reuters.com/news/archive/technologynews?view=page&page=1&pageSize=10'
url2 = 'https://www.reuters.com/news/archive/technologynews?view=page&page=2&pageSize=10'
url3 = 'https://www.reuters.com/news/archive/technologynews?view=page&page=3&pageSize=10'
url4 = 'https://www.reuters.com/news/archive/technologynews?view=page&page=4&pageSize=10'
url5 = 'https://www.reuters.com/news/archive/technologynews?view=page&page=5&pageSize=10'

def web_scraping_news(url):
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html,'lxml')
    div_tag = soup.find_all('h3',attrs={'class':"story-title"})
    news = [i for i in div_tag]

    return news

In [12]:
news1 = web_scraping_news(url1)
news2 = web_scraping_news(url2)
news3 = web_scraping_news(url3)
news4 = web_scraping_news(url4)
news5 = web_scraping_news(url5)
news = news1 + news2 + news3 + news4 + news5
titles = [str(i) for i in news]

#Extract News Headline
headlines = [a.strip(''''<h3 class="story-title">
								''').rstrip('</') for a in titles]

for headline in headlines:
    print(headline)

Elon Musk, back on Twitter, turns his support to Dogecoin
Mazda expects chip shortage to affect about 7,000 vehicles in February
Retail frenzy stalls as focus falls on regulator meeting
GameStop rises, AMC dips in early U.S. premarket trading
Uber's Mideast business Careem sees recovery slowing as infections rise
Australian drone firm reshapes strategy over Google pull-out threat
NatWest latest UK bank to switch to Mastercard debit cards from Visa
Ethereum scales record peak before futures launch
Five things to watch in Reddit stocks trading mania
Taiwan says auto chip shortage not a main topic for coming U.S. meeting
In uneasy truce, House Republicans fail to punish Greene or Cheney
Biden to pursue arms control, seeks to engage China, U.S. envoy says
U.S. charges Seattle-based Proud Boys member for role in Capitol riots
Nokia fourth-quarter profit, revenue beat as CEO Lundmark revamps strategy
Parler CEO John Matze says he was fired by board
Reddit rally' stocks bounce on day after se

### Step 3: Create functions to perform news headlines recommendations

In [13]:
def preprocess(text, char_ngram=5):
    
    return set(text[head:head + char_ngram] for head in range(0, len(text) - char_ngram))

def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['title']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

def predict(text, data, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = data.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [14]:
data = pd.DataFrame(headlines,columns= ['title'])
data.head()

,title
0,"Elon Musk, back on Twitter, turns his support ..."
1,"Mazda expects chip shortage to affect about 7,..."
2,Retail frenzy stalls as focus falls on regulat...
3,"GameStop rises, AMC dips in early U.S. premark..."
4,Uber's Mideast business Careem sees recovery s...


In [15]:
forest = get_forest(data, 100)

It took 0.13052773475646973 seconds to build forest.


### Step 4: Predict news headlines recommendations for the given title.
### "xxxxxxxxxxxxxxxxxxxxxxxx"

In [16]:
title = "Stocks explained: What's going on with GameStop?"
result = predict(title, data, 100, 10, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.004659414291381836 seconds to query forest.

 Top Recommendation(s) is(are) 
 48    Tiktok strengthens content review with new pre...
57    Elon Musk's banter with Robinhood CEO triggers...
59    Sweden's Embracer expands reach with $2.5 bill...
39    Game on after GameStop: Stocks soar again desp...
Name: title, dtype: object
